In [1]:
from ModelsContainer import ModelsContainer
from numpy import average
from energyflow.datasets import qg_jets
from energyflow.utils import data_split, to_categorical
from sklearn.metrics import roc_auc_score
import toploader
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import sys
import os

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print("GPUs Available: ", tf.config.list_physical_devices('GPU'))



/n/home01/rikab/MomentAnalysis/energyflow/archs/__init__.py:30: UserWarning: could not import some architectures - cannot import name 'cnn' from partially initialized module 'energyflow.archs' (most likely due to a circular import) (/n/home01/rikab/MomentAnalysis/energyflow/archs/__init__.py)
  warnings.warn('could not import some architectures - ' + str(e))
/n/home01/rikab/MomentAnalysis/energyflow/archs/__init__.py:40: UserWarning: could not import some architectures - cannot import name 'linear' from partially initialized module 'energyflow.archs' (most likely due to a circular import) (/n/home01/rikab/MomentAnalysis/energyflow/archs/__init__.py)
  warnings.warn('could not import some architectures - ' + str(e))


Num GPUs Available:  0
GPUs Available:  []


2023-08-12 17:43:22.875125: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-08-12 17:43:22.875163: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-08-12 17:43:22.875189: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (boslogin02.rc.fas.harvard.edu): /proc/driver/nvidia/version does not exist


In [2]:
# Parameters 
train = 2
val = 5000
test = 5000
k_order = 1
run_name = "test"
dataset = "qg" # either "qg" or "top"


epochs = 50
batch_size=512
callbacks =None
verbose = 2


num_models_to_train = 3 ##number of models to use to make error bars
order_list = [k_order,] #
input_dim = 2
output_dim = 1


# Directory Handling
base_dir = "/n/home01/rikab/MomentAnalysis/Data"
run_dir = os.path.join(base_dir, run_name)
run_dir = os.path.join(run_dir, f"order_{k_order}")
model_dir = os.path.join(run_dir, "Models")
if not os.path.exists(run_dir):
    os.makedirs(run_dir)
    os.makedirs(model_dir, exist_ok=True)
topdir = "/n/holyscratch01/iaifi_lab/rikab/top"

###########



max_L = 2**(8-k_order)
F_width = 25
Phi_width = 25


Ls = []
j = max_L
while j >= 1:
    Ls.append(j)
    j = j / 2

num_samples = len(Ls)

order_configs = {}
for p, order in enumerate(order_list):


    configs = []
    for i in range(num_samples):
        configs.append([Ls[i], F_width, Phi_width])

    order_configs['Order %d' % order] = np.asarray(configs).astype(np.int32)


In [3]:
if dataset == "qg":
    X, Y = qg_jets.load(train+val+test, cache_dir="/n/holyscratch01/iaifi_lab/rikab/.energyflow")
    X = X[:,:,:3].astype(np.float32)
    for x in X:
        mask = x[:,0] > 0
        yphi_avg = average(x[mask,1:3], weights=x[mask,0], axis=0)
        x[mask,1:3] -= yphi_avg
        x[mask,0] /= x[:,0].sum()


    Y = to_categorical(Y, num_classes=2)

    (z_train, z_val, z_test,
    p_train, p_val, p_test,
    Y_train, Y_val, Y_test) = data_split(X[:,:,0], X[:,:,1:], Y, val=val, test=test)

    X_train = [z_train, p_train]
    X_val = [z_val, p_val]
    X_test = [z_test, p_test]

elif dataset == "top":
    X_train, Y_train = toploader.load(cache_dir=topdir, dataset="train", num_data = train)
    X_test, Y_test = toploader.load(cache_dir=topdir, dataset="test", num_data=test)
    X_val, Y_val = toploader.load(cache_dir=topdir, dataset="val", num_data=test)

    def format(X):

        for x in X:
            mask = x[:,0] > 0
            yphi_avg = average(x[mask,1:3], weights=x[mask,0], axis=0)
            x[mask,1:3] -= yphi_avg
            x[mask,0] /= x[:,0].sum()

        return [X[:,:,0], X[:,:,1:3]]
    
    X_train = format(X_train)
    X_test = format(X_test)
    X_val = format(X_val)

    Y_train = to_categorical(Y_train, num_classes=2)
    Y_test = to_categorical(Y_test, num_classes=2)
    Y_val = to_categorical(Y_val, num_classes=2)


else:
    raise ValueError("dataset must be either `top` or `qg`!")


print("Data loaded!")

Data loaded!


In [6]:
## training

order = 1
L, F, Phi = 1, 100, 100
print(L, F, Phi, order)

callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=8)

model_name = f"O{order}_L{L}_2Phi{Phi}_3F{F}"
container = ModelsContainer(**{'Phi_mapping_dim' : [input_dim,L],
                            'output_dim' : 2, 'output_act' : 'softmax',
                            'Phi_sizes' : [Phi, Phi], 'Phi_acts' : 'ReLU', "Phi_l1_regs" :  1e-7,
                            'F_sizes' : [F,F,F], 'F_acts': 'ReLU', "F_l1_regs" :  1e-7,
                            'order' : order , 'architecture_type':'moment',
                            'loss': 'binary_crossentropy','metrics': 'acc','metrics': ['acc', tf.keras.metrics.AUC()]})
print()
print(i, order, container.num_params)
print()
mean, std = container.train_models(num_models = num_models_to_train,
                            X_train = X_train, Y_train = Y_train,
                            epochs = 5, batch_size = batch_size,
                            path = os.path.join(model_dir , model_name),
                            validation_data = (X_val, Y_val),
                            callbacks=[callback,], verbose=verbose,
                            metric_function = roc_auc_score)
# container.test_meanstd(X_test = X_test, Y_test = Y_test, metric_function = roc_auc_score)
print(mean, std)



1 100 100 1

7 1 31102

Epoch 1/5
1/1 - 3s - loss: 0.7523 - acc: 0.5000 - auc_2: 0.5000 - val_loss: 0.7009 - val_acc: 0.5034 - val_auc_2: 0.5000 - 3s/epoch - 3s/step
Epoch 2/5
1/1 - 1s - loss: 0.7018 - acc: 0.5000 - auc_2: 0.5000 - val_loss: 0.6952 - val_acc: 0.4966 - val_auc_2: 0.5000 - 714ms/epoch - 714ms/step
Epoch 3/5
1/1 - 1s - loss: 0.6949 - acc: 0.5000 - auc_2: 0.5000 - val_loss: 0.7075 - val_acc: 0.4966 - val_auc_2: 0.5000 - 713ms/epoch - 713ms/step
Epoch 4/5
1/1 - 1s - loss: 0.7064 - acc: 0.5000 - auc_2: 0.5000 - val_loss: 0.7133 - val_acc: 0.4966 - val_auc_2: 0.5000 - 1s/epoch - 1s/step
Epoch 5/5
1/1 - 1s - loss: 0.7120 - acc: 0.5000 - auc_2: 0.5000 - val_loss: 0.7097 - val_acc: 0.4966 - val_auc_2: 0.5000 - 1s/epoch - 1s/step
157/157 [==============================] - 1s 7ms/step
Epoch 1/5
1/1 - 2s - loss: 0.8181 - acc: 0.5000 - auc_2: 0.5000 - val_loss: 0.7566 - val_acc: 0.4966 - val_auc_2: 0.5000 - 2s/epoch - 2s/step
Epoch 2/5
1/1 - 1s - loss: 0.7542 - acc: 0.5000 - auc_2: 